**Importing the needed packages**

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from scipy.stats import pearsonr
from deap import base, creator, tools, algorithms
import lightgbm as lgb
import random

In [7]:
#df_monday = pd.read_csv("./data_files/Monday-WorkingHours.pcap_ISCX.csv")
#df_tuesday = pd.read_csv("./data_files/Tuesday-WorkingHours.pcap_ISCX.csv")
df_wednesday = pd.read_csv("./data_files/Wednesday-workingHours.pcap_ISCX.csv")
#df_thursday_afternoon = pd.read_csv("./data_files/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
#df_thursday_morning= pd.read_csv("./data_files/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df_friday_afternoon_ddos = pd.read_csv("./data_files/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df_friday_afternoon_portscan = pd.read_csv("./data_files/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
#df_friday_morning = pd.read_csv("./data_files/Friday-WorkingHours-Morning.pcap_ISCX.csv")
#df_monday.columns = df_monday.columns.str.strip()
#df_tuesday.columns = df_tuesday.columns.str.strip()
df_wednesday.columns = df_wednesday.columns.str.strip()
#df_thursday_afternoon.columns = df_thursday_afternoon.columns.str.strip()
#df_thursday_morning.columns = df_thursday_morning.columns.str.strip()
df_friday_afternoon_ddos.columns = df_friday_afternoon_ddos.columns.str.strip()
df_friday_afternoon_portscan.columns = df_friday_afternoon_portscan.columns.str.strip()
#df_friday_morning.columns = df_friday_morning.columns.str.strip()


In [31]:
df = pd.concat([df_wednesday, df_friday_afternoon_ddos, df_friday_afternoon_portscan], ignore_index=True)
df['Label'] = df['Label'].apply(lambda x: 1 if x == 'BENIGN' else 0)


# Sampling 30% of the data
df = df.sample(frac=0.5, random_state=42)
df['Label'].value_counts()

Label
1    332741
0    269717
Name: count, dtype: int64

In [32]:
# Preprocessing dataset
print("Preprocessing dataset...")

# Step 1: Replace infinite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Step 2: Handle NaN values (e.g., fill with 0 or column mean)
df.fillna(0, inplace=True)

print("Done preprocessing dataset")

Preprocessing dataset...
Done preprocessing dataset


In [42]:
# Step 3: Encode categorical labels
#label_encoder = LabelEncoder()

#df['Label'] = label_encoder.fit_transform(df['Label'])
#print("Labels encoded successfully!")
df['Label'].value_counts()

Label
1    332741
0    269717
Name: count, dtype: int64

In [34]:
# Step 4: Separate features and labels
X = df.drop('Label', axis=1)
Y = df['Label']

In [35]:
# Step 5: Normalize numerical features using MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [36]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [37]:
# Filter Method: Calculate correlations between features and labels
print("Calculating feature importance using correlation...")
correlations = []
for i in range(X_train.shape[1]):
    if np.std(X_train[:, i]) == 0:  # Skip constant features
        correlations.append(0)
    else:
        correlations.append(abs(pearsonr(X_train[:, i], y_train)[0]))
correlation_threshold = 0.2  # Define a threshold to filter irrelevant features
relevant_features = [i for i, corr in enumerate(correlations) if corr > correlation_threshold]
print(f"Features with correlation above threshold: {len(relevant_features)}")

# Subset the data with relevant features only
X_train = X_train[:, relevant_features]
X_test = X_test[:, relevant_features]
feature_names = [df.columns[i] for i in relevant_features]
[print(f"{i}: {feature_names[i]}") for i in range(len(feature_names))]

Calculating feature importance using correlation...
Features with correlation above threshold: 25
0: Destination Port
1: Flow Duration
2: Bwd Packet Length Max
3: Bwd Packet Length Min
4: Bwd Packet Length Mean
5: Bwd Packet Length Std
6: Flow IAT Mean
7: Flow IAT Std
8: Flow IAT Max
9: Fwd IAT Total
10: Fwd IAT Mean
11: Fwd IAT Std
12: Fwd IAT Max
13: Min Packet Length
14: Max Packet Length
15: Packet Length Mean
16: Packet Length Std
17: Packet Length Variance
18: PSH Flag Count
19: URG Flag Count
20: Average Packet Size
21: Avg Bwd Segment Size
22: Idle Mean
23: Idle Max
24: Idle Min


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [39]:
# GA Feature Selection
print("Defining Genetic Algorithm for feature selection...")

# Define GA components
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X_train.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

Defining Genetic Algorithm for feature selection...


In [40]:
# Define the fitness function
def fitness(individual, X_train, y_train):
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:  # Avoid empty feature sets
        return 0,
    X_train_selected = X_train[:, selected_features]
    model = lgb.LGBMClassifier(random_state=42)
    model.fit(X_train_selected, y_train)
    accuracy = model.score(X_train_selected, y_train)  # Training accuracy
    return accuracy,

toolbox.register("evaluate", fitness, X_train=X_train, y_train=y_train)

In [41]:
# Set GA parameters
population = toolbox.population(n=50)
ngen = 40
cxpb = 0.7
mutpb = 0.2

# Run the GA
result_population = algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, verbose=False)
best_individual = tools.selBest(result_population[0], k=1)[0]
selected_features = [i for i, bit in enumerate(best_individual) if bit == 1]

# Print the chosen features
chosen_features = [feature_names[i] for i in selected_features]
print("Selected Features:")
print(chosen_features)

c:\Users\Spencer Kanjera\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 233109, number of negative: 188611
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016463 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3158
[LightGBM] [Info] Number of data points in the train set: 421720, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.552758 -> initscore=0.211819
[LightGBM] [Info] Start training from score 0.211819


c:\Users\Spencer Kanjera\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Spencer Kanjera\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 233109, number of negative: 188611
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025530 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2530
[LightGBM] [Info] Number of data points in the train set: 421720, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.552758 -> initscore=0.211819
[LightGBM] [Info] Start training from score 0.211819


c:\Users\Spencer Kanjera\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Spencer Kanjera\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

[LightGBM] [Info] Number of positive: 233109, number of negative: 188611
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021665 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3558
[LightGBM] [Info] Number of data points in the train set: 421720, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.552758 -> initscore=0.211819
[LightGBM] [Info] Start training from score 0.211819


c:\Users\Spencer Kanjera\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Spencer Kanjera\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 233109, number of negative: 188611
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.069673 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3813
[LightGBM] [Info] Number of data points in the train set: 421720, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.552758 -> initscore=0.211819
[LightGBM] [Info] Start training from score 0.211819


c:\Users\Spencer Kanjera\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Selected Features:
['Destination Port', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Max', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Variance', 'PSH Flag Count', 'URG Flag Count', 'Idle Mean', 'Idle Max']


In [18]:
# Train a model on the selected features
print("Training final model with selected features...")
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]
X_train_selected = pd.DataFrame(X_train_selected, columns=[feature_names[i] for i in selected_features])
X_test_selected = pd.DataFrame(X_test_selected, columns=[feature_names[i] for i in selected_features])

final_model = lgb.LGBMClassifier(
    random_state=42,
    learning_rate=0.05,
    num_leaves=50,
    max_depth=10,
    n_estimators=200,
    boosting_type='gbdt',
    class_weight='balanced',
    verbose_eval= 1  # Print progress every iteration

)
final_model.fit(X_train_selected, y_train)

Training final model with selected features...
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004314 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 622
[LightGBM] [Info] Number of data points in the train set: 253031, number of used features: 4
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from

LGBMClassifier(class_weight='balanced', learning_rate=0.05, max_depth=10,
               n_estimators=200, num_leaves=50, random_state=42,
               verbose_eval=1)

In [19]:
# Evaluate the model
y_pred = final_model.predict(X_test_selected)


# Get feature importance
feature_importance = final_model.feature_importances_

# Combine feature names and their importance into a DataFrame for better readability
feature_importance_df = pd.DataFrame({
    'Feature': X_train_selected.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

[LightGBM] [Warning] Unknown parameter: verbose_eval


In [20]:
# Print the feature importance
print("\nFeature Importance:")
print(feature_importance_df)


Feature Importance:
                  Feature  Importance
3  Init_Win_bytes_forward       48313
1       Min Packet Length       10591
0   Bwd Packet Length Min       10228
2          PSH Flag Count        1759


In [21]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

In [22]:
# Print evaluation metrics
print(f"Final Model Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Final Model Accuracy: 0.8387
Precision: 0.9440
Recall: 0.8387
F1 Score: 0.8745
Confusion Matrix:
[[44837  7179   187   180  5462  1237   663    30]
 [    8 11520     0     2     0     0     0    14]
 [    0     0   269     0   675     0     0     0]
 [    5     0   116 19339  1279     0     0     0]
 [    4     0     0     0   405    44    21     0]
 [    0     0     0     0   321   154     1     0]
 [    0     0     0     0     0     0     1     0]
 [    0     0     0     4    63     0     0 14423]]


In [23]:
#Print a detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86     59775
           1       0.62      1.00      0.76     11544
           2       0.47      0.28      0.35       944
           3       0.99      0.93      0.96     20739
           4       0.05      0.85      0.09       474
           5       0.11      0.32      0.16       476
           6       0.00      1.00      0.00         1
           7       1.00      1.00      1.00     14490

    accuracy                           0.84    108443
   macro avg       0.53      0.77      0.52    108443
weighted avg       0.94      0.84      0.87    108443

